# Redes neuronales: MLP

## 1.- Descripción

Las Redes Neuronales Multicapa (MLP, por sus siglas en inglés: Multilayer Perceptron) son una clase fundamental de modelos dentro del campo de las redes neuronales artificiales. Estas redes consisten en varias capas de neuronas organizadas de forma secuencial, donde cada capa está conectada con la siguiente a través de un conjunto de pesos ajustables.

Las MLP son particularmente efectivas para tareas de clasificación y regresión, ya que pueden aprender representaciones no lineales complejas de los datos. Son modelos supervisados que requieren de un conjunto de datos etiquetado para su entrenamiento. Este tipo de redes es muy popular para tareas de clasificación debido a su capacidad para modelar relaciones complejas entre los datos de entrada y las salidas.

En este caso, se utiliza una Red Neuronal Multicapa (MLP) para predecir si un préstamo será aceptado o rechazado en función de las características del solicitante, como ingresos, edad, historial crediticio, entre otros. Las MLP son adecuadas para este tipo de tareas de clasificación binaria debido a su capacidad para modelar relaciones no lineales complejas entre los datos de entrada y la salida.


## 2.- Implementación


In [942]:
# Importar las librerías necesarias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import StratifiedShuffleSplit
from imblearn.combine import SMOTEENN
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from imblearn.over_sampling import ADASYN
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler



In [943]:
# Cargar datos preprocesados
df = pd.read_csv(r"C:\Users\Day\Downloads\LBBYs-main\LBBYs-main\data\processed\df_train.csv")
df_test= pd.read_csv(r"C:\Users\Day\Downloads\LBBYs-main\LBBYs-main\data\raw\test_nolabel.csv")


In [944]:
# Separar clases mayoritaria y minoritaria
df_majority = df[df["Accept"] == 1]  # Créditos aceptados (clase mayoritaria)
df_minority = df[df["Accept"] == 0]  # Créditos no aceptados (clase minoritaria)

# Aplicar Submuestreo (undersampling)
df_majority_downsampled = resample(df_majority, 
                                   replace=False,  # Sin reemplazo
                                   n_samples=len(df_minority),  # Igualar cantidad de muestras
                                   random_state=42)  # Fijar semilla

# Unir el dataset balanceado
df_balanced = pd.concat([df_majority_downsampled, df_minority])

# Mezclar los datos para evitar sesgos
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)
df.dtypes
# Verificar la distribución de clases en el dataset balanceado
print(df_balanced['Accept'].value_counts())


# Separar características (X) y etiqueta (y)
X = df_balanced.drop('Accept', axis=1)  # Eliminar la columna de la etiqueta 'Accept'
y = df_balanced['Accept']  # La columna de etiqueta

Accept
0    3831
1    3831
Name: count, dtype: int64


In [945]:
df_test_features = df_test.drop(columns=['id'], errors='ignore') 


In [946]:
df_test_features = df_test.copy()

In [947]:
print(df_test.head())

            id  LoanNr_ChkDgt                            Name            City  \
0  6b7ce9ec161     7996984008  SPRINGVALLEY ACCOUNTING SERVIC         HOLLAND   
1  96123015731     3675194008  FULL SCALE DEVELOPMENT CORPORA     WORTHINGTON   
2  b2c5181ac5b     5995734002             KIRBY SAND & GRAVEL  UPPER SANDUSKY   
3  e6cb54a9e6a     7630324004                BLUE ECLIPSE INC          DAYTON   
4  8eddf83466e     4304173001          CAR-PEC GRAPHICS, INC.           AKRON   

  State                            Bank BankState ApprovalDate ApprovalFY  \
0    OH        CITIZENS BANK NATL ASSOC        RI    26-Oct-04       2005   
1    OH    KEYBANK NATIONAL ASSOCIATION        OH     4-Apr-00       2000   
2    OH  JPMORGAN CHASE BANK NATL ASSOC        IL     7-Jan-03       2003   
3    OH  JPMORGAN CHASE BANK NATL ASSOC        IL    21-Jul-04       2004   
4    OH     CASCADE CAPITAL CORPORATION        OH    15-Nov-90       1991   

   NoEmp  NewExist  CreateJob  RetainedJob  Franch

In [948]:
train_columns = X.columns

In [949]:
categorical_columns_test = df_test.select_dtypes(include=['object']).columns

categorical_columns_test = [col for col in categorical_columns_test if col != 'id']
for col in categorical_columns_test:
    if col in categorical_columns:  # Solo transformar las columnas que están en df_train
        # Reemplaza las categorías desconocidas con 'Unknown' si es necesario
        if 'Unknown' not in le.classes_:
            le.classes_ = np.append(le.classes_, 'Unknown')
        
        # Reemplaza las categorías desconocidas con 'Unknown' y aplica la transformación
        df_test[col] = df_test[col].astype(str).apply(lambda x: x if x in le.classes_ else 'Unknown')
        df_test[col] = le.transform(df_test[col])

In [950]:
print(df_test.head())

            id  LoanNr_ChkDgt  Name  City  State  Bank  BankState  \
0  6b7ce9ec161     7996984008     1     1      1     1          1   
1  96123015731     3675194008     1     1      1     1          1   
2  b2c5181ac5b     5995734002     1     1      1     1          1   
3  e6cb54a9e6a     7630324004     1     1      1     1          1   
4  8eddf83466e     4304173001     1     1      1     1          1   

   ApprovalDate ApprovalFY  NoEmp  NewExist  CreateJob  RetainedJob  \
0             1       2005      9       1.0          0            9   
1             1       2000     15       1.0          4           15   
2             1       2003     13       1.0          0           13   
3             1       2004     28       2.0         19           28   
4             1       1991     20       1.0         17            3   

   FranchiseCode  UrbanRural RevLineCr LowDoc  DisbursementDate  \
0              1           2         Y      N                 1   
1              1        

In [951]:
le = LabelEncoder()

# Se aplica a todas las columnas categóricas que contienen texto
categorical_columns = X.select_dtypes(include=['object']).columns
for col in categorical_columns:
    X[col] = le.fit_transform(X[col].astype(str))


In [952]:
categorical_columns_test = df_test_features.select_dtypes(include=['object']).columns
le = LabelEncoder()

for col in categorical_columns_test:
    df_test_features[col] = le.fit_transform(df_test_features[col].astype(str))


In [953]:
# Imputar valores faltantes con la media (solo para datos numéricos)
imputer = SimpleImputer(strategy='mean')
df_test_imputed = imputer.fit_transform(df_test_features)

In [954]:
# Escalar las características
scaler = StandardScaler()
df_test_scaled = scaler.fit_transform(df_test_imputed)

In [955]:
# Imputar valores faltantes con la media (puedes cambiar la estrategia a 'median' o 'most_frequent')

imputer = SimpleImputer(strategy='mean')  # Para valores numéricos
X_imputed = imputer.fit_transform(X)

In [956]:
# Escalar las características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

In [1090]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

## 3.- Evaluación y optimización


In [1093]:
# Crear y entrenar el modelo MLP
mlp = MLPClassifier(
    hidden_layer_sizes=(300, 200, 100, 50), 
    activation='relu', 
    solver='adam', 
    max_iter=9000, 
    random_state=42,
    learning_rate_init=0.001,
    alpha=0.0001,
    batch_size='auto',
    momentum=0.9,
    n_iter_no_change=10,
    early_stopping=True,
    validation_fraction=0.1,
    tol=1e-4
)

mlp.fit(X_train, y_train)

MLPClassifier(early_stopping=True, hidden_layer_sizes=(300, 200, 100, 50),
              max_iter=9000, random_state=42)

In [1094]:
# Realizar las predicciones
y_pred = mlp.predict(X_test)

In [1095]:
# Hacer predicciones
y_test_pred = mlp.predict(df_test_scaled)

In [1096]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[708 427]
 [326 838]]
              precision    recall  f1-score   support

           0       0.68      0.62      0.65      1135
           1       0.66      0.72      0.69      1164

    accuracy                           0.67      2299
   macro avg       0.67      0.67      0.67      2299
weighted avg       0.67      0.67      0.67      2299



In [1097]:
df_test['ACCEPT'] = y_test_pred
print(df_test[['id', 'ACCEPT']].head())
missing_accept = df_test['ACCEPT'].isnull().sum()
print(missing_accept)

            id  ACCEPT
0  6b7ce9ec161       0
1  96123015731       0
2  b2c5181ac5b       0
3  e6cb54a9e6a       1
4  8eddf83466e       1
0


In [1098]:
# Crear DataFrame con las predicciones (id y Accept)
df_result = df_test[['id']].copy()  # Agregar 'id' del conjunto de test
df_result['ACCEPT'] = y_test_pred
# Verifica las primeras filas del DataFrame
print(df_result.head())

            id  ACCEPT
0  6b7ce9ec161       0
1  96123015731       0
2  b2c5181ac5b       0
3  e6cb54a9e6a       1
4  8eddf83466e       1


In [1099]:
# Guardar el resultado en un archivo CSV
df_result.to_csv("SUBMMIT_MLP.csv", index=False)

print("¡Predicciones completas y archivo guardado como 'resultado_test.csv'!")

¡Predicciones completas y archivo guardado como 'resultado_test.csv'!


In [1100]:
# Para los resultados de entrenamiento
y_train_pred = mlp.predict(X_train)
print("Reporte de clasificación para el entrenamiento:")
print(classification_report(y_train, y_train_pred))

# Para los resultados de prueba
y_test_pred = mlp.predict(X_test)
print("Reporte de clasificación para la prueba:")
print(classification_report(y_test, y_test_pred))

Reporte de clasificación para el entrenamiento:
              precision    recall  f1-score   support

           0       0.72      0.70      0.71      2696
           1       0.70      0.73      0.72      2667

    accuracy                           0.71      5363
   macro avg       0.71      0.71      0.71      5363
weighted avg       0.71      0.71      0.71      5363

Reporte de clasificación para la prueba:
              precision    recall  f1-score   support

           0       0.68      0.62      0.65      1135
           1       0.66      0.72      0.69      1164

    accuracy                           0.67      2299
   macro avg       0.67      0.67      0.67      2299
weighted avg       0.67      0.67      0.67      2299



In [1088]:
from sklearn.model_selection import cross_val_score
# Evaluación utilizando validación cruzada
cv_scores = cross_val_score(mlp, X_train, y_train, cv=5, scoring='accuracy')
print(f"Cross-validated accuracy: {cv_scores.mean()} ± {cv_scores.std()}")

Cross-validated accuracy: 0.6538727060433481 ± 0.013811870942865966


In [1109]:
# Comprobar las dimensiones de X_train y df_test_scaled
print("Dimensiones de X_train:", X_train.shape)
print("Dimensiones de df_test_scaled:", df_test_scaled.shape)

Dimensiones de X_train: (5363, 20)
Dimensiones de df_test_scaled: (3284, 20)


## 4.- Conclusión

El modelo MLP muestra un rendimiento equilibrado, con una precisión del 67% para los préstamos rechazados (clase 0) y del 70% para los aceptados (clase 1). El recall es similar en ambas clases, lo que indica que el modelo tiene una capacidad decente para identificar correctamente ambos tipos de préstamos, aunque no es perfecto. La precisión global del modelo es del 68%, lo que sugiere un rendimiento moderado en la predicción de las decisiones de préstamo.
